In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm import tqdm

In [3]:
#---------------------------------------------------------------------------
#target : find the list of the names from wikipedia, start from letter A by pressing next page iterate.
correct_names = []#list of names.
url =f"https://en.wikipedia.org/wiki/Category:Given_names?from=A"
while(True):#run till we get False of next page.
  next_page = False#our flag
  response = requests.get(url)
  soup = BeautifulSoup(response.content,"html.parser")
  bodyDiv = soup.find("div",{"id":"mw-pages"})
  namesDiv = bodyDiv.find("div",{"class":"mw-category mw-category-columns"})
  names = namesDiv.findAll("a")#save the names for the iteration.
  for name in names:#make our list correct.
    correct_names.append(name.text)
  bodyDiv = soup.find("div",{"id":"mw-pages"})
  names = bodyDiv.findAll("a")
  for name_link in names:#run through the links of the names till we get FALSE from flag.
    if (name_link.text == 'next page'):
      next_page = True
      url = "https://en.wikipedia.org" + name_link['href']#if there is no false then start the next page.
  if next_page == False:
    break

In [4]:
len(correct_names)

19464

In [5]:
#---------------------------------------------------------------------------
Qֹֹֹ_and_name = []#name that has wikidata value
page_ids = []#list of Q_ID for wikiData
#---------------------------------------------------------------------------
page_titles = list(correct_name for correct_name in correct_names )
for page_title in tqdm(page_titles):
  api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={page_title}&prop=pageprops&format=json'
  response = requests.get(api_url).json()
  try:#if the name has id then we save it or else we pass.
    page_id = next(iter(response['query']['pages'].values()))['pageprops']['wikibase_item']
    page_ids.append(page_id)
    Qֹֹֹ_and_name.append(page_title)
  except:
    pass

100%|██████████| 19464/19464 [25:44<00:00, 12.60it/s]


In [9]:
len(page_ids)

17358

In [7]:
def func(page_ids,x,name_index):
  #------------------------------------------------------------------------------
  table = []#the final list of the rows
  # name_index = 0#index for running through the name list
  #------------------------------------------------------------------------------
  #here we make the final list with rows for every Q_number
  # page_ids = ['Q21502297']
  list_ex1 = []
  for Q_number in tqdm(page_ids):
    try:
      url = f"https://www.wikidata.org/wiki/{Q_number}"
      response = requests.get(url)
      soup = BeautifulSoup(response.content,"html.parser")
      enteries = soup.findAll("div",{"class":"mw-collapsible-content"})
      li_list = enteries[0].findAll("li")
      count = 0
      # print(li_list)
      new_span_c = []
      for li in li_list :#creat the rows of the table
        span_site_link = li.findAll("span",{"class":"wikibase-sitelinkview-page"})
        span = soup.findAll("span",{"class":"wikibase-descriptionview-text"})
        container = enteries[0].findAll("span",{"class":"wikibase-sitelinkview-siteid-container"})
        if len(new_span_c) == 0:
          for i in range(len(container)):
            span_continer = container[i].findAll("span")
            new_span_c.append(span_continer[0])
        table.append([Qֹֹֹ_and_name[name_index],Q_number,span[0].getText(),new_span_c[0].get("title"),span_site_link[0].get("lang"),span_site_link[0].getText()])
        new_span_c.pop(0)
    except:
      print(Q_number)
      list_ex1.append(Q_number)
    name_index+=1

  #------------------------------------------------------------------------------
  #creat the table
  results_df = pd.DataFrame(table, columns=['Label','WikiDate ID', 'English Description','Language','Wiki Short Lang','Entry'])
  # results_df = results_df[['Label','WikiDate ID', 'English Description','Language','Wiki Short Lang','Entry']]#change the coulmns
  results_df#print, it is our test 
  results_df.to_csv('Given_names'+x+'.csv', index=False, encoding='utf-8-sig')
  return name_index
  # conc.append(results_df)

In [11]:
len(page_ids)

17358

In [12]:
name_index = 0

In [13]:
index = func(page_ids[:10000],'1',name_index)


100%|██████████| 10000/10000 [17:50<00:00,  9.34it/s]


In [8]:
index = func(page_ids[10000:],'2',index)
print(index)

100%|██████████| 7358/7358 [12:40<00:00,  9.67it/s]


17358


In [9]:
t1 = pd.read_csv('Given_names1.csv')#0-1
t2 = pd.read_csv('Given_names2.csv')#1-2

big_table = [t1,t2]
big_table_df = pd.concat(big_table,ignore_index = True)


In [ ]:
# We had to split the run into two parts because
#  the number of names with Q_ids is 17358 and  Google Colab
#  cannot handle a long hour run.

In [ ]:
#we use concat to create the big table


In [10]:
big_table_df

,Label,WikiDate ID,English Description,Language,Wiki Short Lang,Entry
0,Aad,Q300770,male given name,German,de,Aad
1,Aad,Q300770,male given name,English,en,Aad
2,Aad,Q300770,male given name,Persian,fa,آد
3,Aad,Q300770,male given name,Chinese,zh,阿德
4,Aadil,Q96476141,masculine given name,English,en,Aadil
...,...,...,...,...,...,...
44198,Zygmunt,Q245304,male given name,Polish,pl,Zygmunt
44199,Þórir,Q14312096,Wikimedia disambiguation page,English,en,Þórir
44200,Þórir,Q14312096,Wikimedia disambiguation page,Italian,it,Thorir
44201,Þorlákur,Q3482049,male given name,English,en,Þorlákur


In [12]:
big_table_df.to_csv('final_Given_names.csv', index=False, encoding='utf-8-sig')
